In [1]:
from qiskit import IBMQ
# IBMQ.save_account(MY_API_TOKEN)
import qiskit 
qiskit.__version__

import numpy as np
import random
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.quantum_info.operators import Operator
from qiskit import(QuantumCircuit, execute, Aer)
from qiskit.visualization import plot_histogram
from qiskit.extensions import Initialize # Import the Inititialize function
from qiskit.aqua.circuits.gates import multi_control_toffoli_gate

In [2]:
visible = 2
hidden = 2

x = np.array([random.uniform(0, 1) for n in range(visible)])
weight_matrix = np.random.rand(visible, hidden) * np.pi
# [[np.pi/6, np.pi/4], [np.pi / 2, np.pi]]


In [3]:
qc = QuantumCircuit(visible + hidden, visible)
initial_state = [[np.sqrt(x[i]), np.sqrt(1-x[i])] for i in range(len(x))]
for i in range(visible):
    initialize_qubit = Initialize(initial_state[i])
    qc.append(initialize_qubit, [i])
    qc.h(i)
for i in range(hidden):
    for j in range(visible):
        qc.rz(weight_matrix[j][i], j)
#     print(list(range(visible)) + [visible + i])
    multi_control_toffoli_gate._multicx(qc, list(range(visible)) + [visible + i])
for i in range(hidden):
    qc.h(visible + i)
# Map the quantum measurement to the classical bits
qc.measure(list(range(visible)), list(range(visible)))

In [4]:
# Use Aer's qasm_simulator
simulator = Aer.get_backend('qasm_simulator')

# Execute the circuit on the qasm simulator
job = execute(qc, simulator, shots=1000)

# Grab results from the job
result = job.result()

# Returns counts
counts = result.get_counts(qc)
print("\nTotal count for 0 and 1 are:",counts)
qc.draw()


Total count for 0 and 1 are: {'01': 4, '10': 65, '00': 931}


┌────────────────────────────┐┌───┐ ┌────────────┐     ┌────────────┐»
q_0: |0>─┤ Initialize(0.72995,0.6835) ├┤ H ├─┤ Rz(2.8161) ├──■──┤ Rz(2.4315) ├»
        ┌┴────────────────────────────┤├───┤┌┴────────────┤  │  ├────────────┤»
q_1: |0>┤ Initialize(0.86164,0.50753) ├┤ H ├┤ Rz(0.32826) ├──■──┤ Rz(1.3743) ├»
        └─────────────────────────────┘└───┘└─────────────┘┌─┴─┐└───┬───┬────┘»
q_2: |0>───────────────────────────────────────────────────┤ X ├────┤ H ├─────»
                                                           └───┘    └───┘     »
q_3: |0>──────────────────────────────────────────────────────────────────────»
                                                                              »
 c_0: 0 ══════════════════════════════════════════════════════════════════════»
                                                                              »
 c_1: 0 ══════════════════════════════════════════════════════════════════════»
                                                                              »
«               ┌─┐   
«q_0: ──■───────┤M├───
«       │       └╥┘┌─┐
«q_1: ──■────────╫─┤M├
«       │        ║ └╥┘
«q_2: ──┼────────╫──╫─
«     ┌─┴─┐┌───┐ ║  ║ 
«q_3: ┤ X ├┤ H ├─╫──╫─
«     └───┘└───┘ ║  ║ 
«c_0: ═══════════╩══╬═
«                   ║ 
«c_1: ══════════════╩═
«

In [5]:
# visible = 2
# hidden = 2
# qc = QuantumCircuit(visible+hidden+ancilla, visible+hidden)
# for i in range(visible+ hidden):
#     initialize_qubit = Initialize([0, 1])
#     qc.append(initialize_qubit, [i])
# # initialize_qubit = Initialize([1, 0])
# # qc.append(initialize_qubit, [2])
# # for i in range(hidden):
# #     for j in range(visible):
# #         qc.rz(weight_matrix[j][i], j)
# print(list(range(visible)) + [visible + 0])
# multi_control_toffoli_gate._multicx(qc, list(range(visible)) + [visible + 0])
# qc.measure(list(range(visible+hidden)), list(range(visible + hidden)))



# # Use Aer's qasm_simulator
# simulator = Aer.get_backend('qasm_simulator')

# # Execute the circuit on the qasm simulator
# job = execute(qc, simulator, shots=1000)

# # Grab results from the job
# result = job.result()

# # Returns counts
# counts = result.get_counts(qc)
# print("\nTotal count for 0 and 1 are:",counts)
# qc.draw()
# qc.draw()

In [6]:
print(weight_matrix)
sinus_matrix = np.sin(weight_matrix + np.pi/4)
print(sinus_matrix)
print(sinus_matrix.prod(axis = 0))
phi = np.arcsin(sinus_matrix.prod(axis = 0))
print(phi)
psi = phi + np.pi/4
print(psi)
ph = np.sin(psi)**2
print(ph)

[[2.8161375  2.43150441]
 [0.32825671 1.37429235]]
[[-0.44389704 -0.07523876]
 [ 0.89731788  0.8315554 ]]
[-0.39831675 -0.06256519]
[-0.40968101 -0.06260608]
[0.37571716 0.72279208]
[0.13464478 0.43755738]


In [ ]:
# template do uczenia

def update(weight_matrix, h_probabilities, exp_h_probabilities):
    ...

num_epochs = 7
weight_matrix = np.random.rand(visible, hidden) * np.pi

for i in range(num_epochs):
    qc = QuantumCircuit(visible + hidden, visible)
    initial_state = [[np.sqrt(x[i]), np.sqrt(1-x[i])] for i in range(len(x))]
    for i in range(visible):
        initialize_qubit = Initialize(initial_state[i])
        qc.append(initialize_qubit, [i])
        qc.h(i)
    for i in range(hidden):
        for j in range(visible):
            qc.rz(weight_matrix[j][i], j)
    #     print(list(range(visible)) + [visible + i])
        multi_control_toffoli_gate._multicx(qc, list(range(visible)) + [visible + i])
    for i in range(hidden):
        qc.h(visible + i)
    # Map the quantum measurement to the classical bits
    qc.measure(list(range(visible)), list(range(visible)))
    
    simulator = Aer.get_backend('qasm_simulator')

    # Execute the circuit on the qasm simulator
    job = execute(qc, simulator, shots=1000)

    # Grab results from the job
    result = job.result()

    # Returns counts
    counts = result.get_counts(qc)
    print("\nTotal count for 0 and 1 are:",counts)
    qc.draw()
    
    h_probabilities = []
    exp_h_probabilities = []
    
    update(weight_matrix, h_probabilities, exp_h_probabilities)
    